In [16]:
## CLOT

#Beautiful Soup is used for parsing the html code and finding elements within it to properly extract key pieces of data
from bs4 import BeautifulSoup as soup
#Im using splinter to navigate the browser with an automated testing system, logging in, click buttons, and such...
from splinter import Browser
# Importing Pandas to read the CSV files
import pandas as pd
#==== May not need whatever is below =====
# Importing date-time
import datetime as datetime

# importing time so Python can sleep, making net32 think we are not launching a denial of service attack
import time 

# The python event scheduler
import schedule 


### Eastern Time Class ###

class EST5EDT(datetime.tzinfo):

    def utcoffset(self, dt):
        return datetime.timedelta(hours=-5) + self.dst(dt)

    def dst(self, dt):
        d = datetime.datetime(dt.year, 3, 8)        #2nd Sunday in March
        self.dston = d + datetime.timedelta(days=6-d.weekday())
        d = datetime.datetime(dt.year, 11, 1)       #1st Sunday in Nov
        self.dstoff = d + datetime.timedelta(days=6-d.weekday())
        if self.dston <= dt.replace(tzinfo=None) < self.dstoff:
            return datetime.timedelta(hours=1)
        else:
            return datetime.timedelta(0)
        
###################################

executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome',**executable_path, headless = False)

monitor = 'Y'

In [17]:
while monitor == 'Y':

    
    currentTime = datetime.datetime.now(tz = EST5EDT()).strftime('%H:%M:%S')
    
    if currentTime >= '23:00:00':
        monitor = "N"
    else:
        print(currentTime)

        
url = "https://juicestore.com/"

browser.visit(url)

# Rendering the HTML Shop page 
html = browser.html
page_soup = soup(html, 'html.parser')

time.sleep(1.5)
### CLicking the Account button to log into my account 
browser.find_by_text('Account').click()

### Filling in log-in details
## Creating XML paths for the browser to search through the elements i want to fill with login details
logInPath = "//input[@id = 'CustomerEmail']"
passwordPath = "//input[@id = 'CustomerPassword']"
## Creating XML path for the browser to click the login button
submitPath = "//input[@value = 'Login']"

######################################
######################################
######################################
## Having the browser fill in my email

browser.find_by_xpath(logInPath).fill('marc.mayuyu@gmail.com')

time.sleep(1.5)

## Having the browser fill in my password
browser.find_by_xpath(passwordPath).fill('Ju1c3St0r3Pw6')

time.sleep(1.5)
######################################
######################################
######################################


## Hving the browser click the submit button
browser.find_by_xpath(submitPath).click()

time.sleep(2)

### Once on the account page, navigate to the shop page 
## Clicking the shop page by text
browser.find_by_text("Shop").click()

time.sleep(1.5)

### Direct the browser to the Clot section of the store 
browser.visit('https://www.juicestore.com/collections/new-arrival')


# Rendering the HTML new arrival page 
html = browser.html
page_soup = soup(html, 'html.parser')


## This bit of code brings me to the exact href for each individual clothing article, so i can collect the href's for each item
gridOfClothes = page_soup.find_all('div', class_ = 'medium-up--three-quarters')[0].find_all('div', class_ = 'grid__item')[1].find_all('div', class_ = 'grid')[0]

## This is the amount of clothing articles on the current html page we are on
### Disclaimer: Since the site automatically filters it's clothing articles to 'latest', i feel as though it's not necessary
## to collect all values of all clothes in other pages, and just to focus on the first page based on how the filter for the 
## store behaves
amountOfArticlesOnPage = len(gridOfClothes.find_all('div', class_ = 'grid__item'))


### For all the clothing articles on the page, extract all their href's and their 
for i in range(0,amountOfArticlesOnPage):
    clothingArticleHref = gridOfClothes.find_all('div', class_ = 'grid__item')[i].find_all('div', class_ = 'product-card')[0].find_all('a', class_ = 'grid-view-item__link')[0]['href']


    ### Removing all 27 chars from the string, as the pattern for all cothing article holds these first 27 characters, and the following change
    ## this allows me to focus on the difference between items when looking for a particular item
    removingFirst27 = clothingArticleHref[27:]


    ##Ater removing thre first 27 characters in the URL string, i'm going to replace all "-" from the url with a space, so i can 
    ## pinpoint key words in a string
    string = removingFirst27.replace("-"," ")

    string = string.lower()

    if "rose" in string and "gold" in string and "air" in string or "force" in string:
        print("its in this href")
        print(clothingArticleHref)        
        browser.click_link_by_href(clothingArticleHref)
        break
              
### Selecting the shoes

## Rendering the HTML Shop page 
html = browser.html
page_soup = soup(html, 'html.parser')

containerWSizeQtyCart= page_soup.find_all("form", id = "product_form_4373019000890")[0]

sizePick = 'w us 9'

## Finding the amount of shoe sizes available
amtSizeAvail = len(containerWSizeQtyCart.find_all("div", class_ = "product-option")[0].find_all("div", class_ = "product-option__option"))

## For every 'size' available, have them grouped and ready to be indexed
size = containerWSizeQtyCart.find_all("div", class_ = "product-option")[0].find_all("div", class_ = "product-option__option")

### For every size available, iterate through the range of sizes available
for i in range(0,amtSizeAvail):
    ## If the size available is the same as the size i picked (I want)
    if size[i].text.strip().lower() == sizePick:        
        ## click on the size and then break the loop so i can navigate to the check out page
        browser.find_by_text(size[i].text.strip()).click()
        break

time.sleep(1.5)
### Add to cart
addToCartPath = "//button[@id = 'AddToCart-product-template']"
browser.find_by_xpath(addToCartPath).click()

time.sleep(1.5)

### CLicking the Check Out Button
browser.find_by_css('.btn__checkout')[0].click()

## On the shipping address page
# Going to select my address from the dropdown menu right under "Shipping Address"

# finding the drop down menu by xpath
dropDownPath = "//select[@data-trekkie-id = 'shipping_address_field']"
browser.find_by_xpath(dropDownPath)[0].click()

## clicking my address on the drop down menu by xpath
address = "//option[@value = '3047406338106']"
browser.find_by_xpath(address).click()
browser.find_by_xpath(dropDownPath)[0].click()

### CLicking the Continue to Shipping button
## Clicking the browser by finding the CSS (Class)
browser.find_by_css('.step__footer__continue-btn').click()

### Clicking the 'Continue To Payment' button
paymentPath = "//button[@data-trekkie-id = 'continue_to_payment_method_button']"
browser.find_by_xpath(paymentPath).click()


## Rendering the HTML Shop page 
html = browser.html
page_soup = soup(html, 'html.parser')

### On the payment page
## Filling out the card numebr field
cardNumberPath = "//iframe[@class = 'card-fields-iframe']"
browser.find_by_xpath(cardNumberPath).click()

IndexError: list index out of range